In [ ]:
## This file implements neural networks for p003pkpresabs_qual with four replicates.
## We compute the mean and standarad deviation of training and test accuracies.
## We also compute the mean and standard deviation of AUC ROC values for four replicates.

In [54]:
from numpy.random import seed
import numpy as np
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [55]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/p003pkpresabs_qual.csv')
df.shape

(253, 868)

In [56]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [57]:
df['pheno']

0      0
1      0
2      1
3      0
4      0
      ..
248    0
249    0
250    0
251    0
252    0
Name: pheno, Length: 253, dtype: int64

In [58]:
df.head()

,id,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATG,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGA,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAA,TTTTTTCTTTTCATAACTGTGTTGGAAATGAATTAAATTAACAGCTCTTTGTGCTTTACGGTGTGTTGC,TTTTTTCAGCATTGTCTACATTACTTAACATTCGTGTTTGTAAGTAATATTGACCGCCAATATTTAGACACTTTATAAGTATGCCATTCATCATTTTTAA,TTTTTTATCTCACCAATTTTGTAATACATCGTTCTCGTCCTCCTTGTCTTCTTCGTCCTCCTCGTTATCTTCTTCGTTTTGTAATTCATAAATTTTGTTT,TTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAAA,TTTTTTAGGTACC,TTTTTGCATTCA,...,AAGTATGTTTTT,AAGTAAAGGTTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGATAAGTAATCTTAATGCTTCAAAAGATTT,AACTAGGGGGGATTAGAATGCAAAATAAGTCAAAATCGCCTTTTAAAATTGCATTTTCTAAATTTATTCATAATAAAATTGCAATGTTATCGGTTATTTT,AACAAACTCAATTAATACGAATTATCGCTTTCAATAAAAATTATTCATTAAATCATTAAAGATATTGAGTTCCAATACTATTTTCACTTTTCATTAATTT,AAATTTTTCAACAAACTCAATTAATACGAATTATCGCTTTCAATAAAAATTATTCATTAAATCATTAAAGATATTGAGTTCCAATACTATTTTCACTTTT,AAATTTTACTTAACAACTAGTACATAAAGTAATACAATTAAATTAATTCTATCTGAAAGATGTGTGGGGCATCGTTATTTTAGGTGGATATGAGCAATTT,AAATTGCGTATTT,AAATCTTAGACGTAAACATGATAAAATGGCCTTGATTACTCAATAGTTATATTTCGGAGAACTGATTTGTGATATGATATTAAAGACTATAGGAGGATTT,AAATCTACTGTTT,pheno
0,107,1,1,1,0,0,0,1,1,1,...,1,1,0,0,0,0,0,0,1,0
1,109,1,1,1,0,0,0,1,1,1,...,1,1,0,0,0,0,0,0,1,0
2,115,1,1,1,1,1,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,120335,1,1,1,0,0,0,1,1,1,...,1,1,0,0,0,0,0,0,1,0
4,120337,1,1,1,0,0,0,1,1,1,...,1,1,0,0,0,0,0,0,1,0


In [59]:
df['pheno'].value_counts()

0    224
1     26
2      3
Name: pheno, dtype: int64

In [60]:
df_clean = df.drop(columns=['id'])

In [61]:
df_clean.shape

(253, 867)

In [62]:
df_clean.head()

,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATG,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGA,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAA,TTTTTTCTTTTCATAACTGTGTTGGAAATGAATTAAATTAACAGCTCTTTGTGCTTTACGGTGTGTTGC,TTTTTTCAGCATTGTCTACATTACTTAACATTCGTGTTTGTAAGTAATATTGACCGCCAATATTTAGACACTTTATAAGTATGCCATTCATCATTTTTAA,TTTTTTATCTCACCAATTTTGTAATACATCGTTCTCGTCCTCCTTGTCTTCTTCGTCCTCCTCGTTATCTTCTTCGTTTTGTAATTCATAAATTTTGTTT,TTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAAA,TTTTTTAGGTACC,TTTTTGCATTCA,TTTTTGAAAATAATCATTAGCTTGCTCACTATATAATTTGATGAATATATTTCGTGAAAGTGGGTATTTATTTAATGATTATTCTATATATGATAGTATA,...,AAGTATGTTTTT,AAGTAAAGGTTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGATAAGTAATCTTAATGCTTCAAAAGATTT,AACTAGGGGGGATTAGAATGCAAAATAAGTCAAAATCGCCTTTTAAAATTGCATTTTCTAAATTTATTCATAATAAAATTGCAATGTTATCGGTTATTTT,AACAAACTCAATTAATACGAATTATCGCTTTCAATAAAAATTATTCATTAAATCATTAAAGATATTGAGTTCCAATACTATTTTCACTTTTCATTAATTT,AAATTTTTCAACAAACTCAATTAATACGAATTATCGCTTTCAATAAAAATTATTCATTAAATCATTAAAGATATTGAGTTCCAATACTATTTTCACTTTT,AAATTTTACTTAACAACTAGTACATAAAGTAATACAATTAAATTAATTCTATCTGAAAGATGTGTGGGGCATCGTTATTTTAGGTGGATATGAGCAATTT,AAATTGCGTATTT,AAATCTTAGACGTAAACATGATAAAATGGCCTTGATTACTCAATAGTTATATTTCGGAGAACTGATTTGTGATATGATATTAAAGACTATAGGAGGATTT,AAATCTACTGTTT,pheno
0,1,1,1,0,0,0,1,1,1,0,...,1,1,0,0,0,0,0,0,1,0
1,1,1,1,0,0,0,1,1,1,0,...,1,1,0,0,0,0,0,0,1,0
2,1,1,1,1,1,0,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,1,1,1,0,0,0,1,1,1,0,...,1,1,0,0,0,0,0,0,1,0
4,1,1,1,0,0,0,1,1,1,0,...,1,1,0,0,0,0,0,0,1,0


In [63]:
X = df.loc[:, df.columns != 'pheno']
y = df['pheno']
print(X.shape, y.shape)

(253, 867) (253,)


In [64]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 224), (1, 224), (2, 224)]


/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [65]:
############# Fully-Connected Neural Network ################

In [66]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1

In [67]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [68]:
dat = pd.DataFrame(X_test_over[:,0])
dat['test'] = y_test_over

In [69]:
dat

,0,test
0,NRS265,1
1,GA984,0
2,NRS119,0
3,NRS249,1
4,NRS255,2
...,...,...
197,NRS035,1
198,NRS387,1
199,NRS222,0
200,BCH-SA-11,1


In [70]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [18]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [19]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 470 samples, validate on 202 samples
Epoch 1/1000
470/470 [==============================] - 0s 314us/step - loss: 1.2874 - accuracy: 0.4213 - val_loss: 0.8638 - val_accuracy: 0.6089
Epoch 2/1000
470/470 [==============================] - 0s 106us/step - loss: 0.7310 - accuracy: 0.6447 - val_loss: 0.6236 - val_accuracy: 0.7079
Epoch 3/1000
470/470 [==============================] - 0s 108us/step - loss: 0.6273 - accuracy: 0.7191 - val_loss: 0.5473 - val_accuracy: 0.8020
Epoch 4/1000
470/470 [==============================] - 0s 93us/step - loss: 0.5520 - accuracy: 0.8085 - val_loss: 0.5086 - val_accuracy: 0.7673
Epoch 5/1000
470/470 [==============================] - 0s 104us/step - loss: 0.5074 - accuracy: 0.7745 - val_loss: 0.4767 - val_accuracy: 0.8168
Epoch 6/1000
470/470 [==============================] - 0s 125us/step - loss: 0.4629 - accuracy: 0.8191 - val_loss: 0.4558 - val_accuracy: 0.7673
Epoch 7/1000
470/470 [==============================] - 0s 94us/step - loss: 0.

470/470 [==============================] - 0s 106us/step - loss: 0.2110 - accuracy: 0.9298 - val_loss: 0.3807 - val_accuracy: 0.8366
Epoch 113/1000
470/470 [==============================] - 0s 98us/step - loss: 0.2089 - accuracy: 0.9106 - val_loss: 0.3291 - val_accuracy: 0.8663
Epoch 114/1000
470/470 [==============================] - 0s 119us/step - loss: 0.2051 - accuracy: 0.9149 - val_loss: 0.3219 - val_accuracy: 0.8713
Epoch 115/1000
470/470 [==============================] - 0s 123us/step - loss: 0.2010 - accuracy: 0.9234 - val_loss: 0.3278 - val_accuracy: 0.8762
Epoch 116/1000
470/470 [==============================] - 0s 102us/step - loss: 0.2046 - accuracy: 0.9213 - val_loss: 0.3495 - val_accuracy: 0.8564
Epoch 117/1000
470/470 [==============================] - 0s 105us/step - loss: 0.1991 - accuracy: 0.9149 - val_loss: 0.3278 - val_accuracy: 0.8663
Epoch 118/1000
470/470 [==============================] - 0s 107us/step - loss: 0.1956 - accuracy: 0.9234 - val_loss: 0.3176 - v

470/470 [==============================] - 0s 86us/step - loss: 0.1953 - accuracy: 0.9191 - val_loss: 0.4214 - val_accuracy: 0.8416
Epoch 223/1000
470/470 [==============================] - 0s 114us/step - loss: 0.1761 - accuracy: 0.9255 - val_loss: 0.3334 - val_accuracy: 0.8762
Epoch 224/1000
470/470 [==============================] - 0s 92us/step - loss: 0.1824 - accuracy: 0.9191 - val_loss: 0.3436 - val_accuracy: 0.8663
Epoch 225/1000
470/470 [==============================] - 0s 92us/step - loss: 0.1973 - accuracy: 0.9234 - val_loss: 0.3855 - val_accuracy: 0.8416
Epoch 226/1000
470/470 [==============================] - 0s 85us/step - loss: 0.1921 - accuracy: 0.9213 - val_loss: 0.3333 - val_accuracy: 0.8812
Epoch 227/1000
470/470 [==============================] - 0s 84us/step - loss: 0.1939 - accuracy: 0.9191 - val_loss: 0.3291 - val_accuracy: 0.8713
Epoch 228/1000
470/470 [==============================] - 0s 85us/step - loss: 0.2175 - accuracy: 0.9170 - val_loss: 0.3792 - val_ac

Epoch 278/1000
470/470 [==============================] - 0s 89us/step - loss: 0.1719 - accuracy: 0.9255 - val_loss: 0.3845 - val_accuracy: 0.8465
Epoch 279/1000
470/470 [==============================] - 0s 94us/step - loss: 0.1723 - accuracy: 0.9149 - val_loss: 0.3329 - val_accuracy: 0.8762
Epoch 280/1000
470/470 [==============================] - 0s 87us/step - loss: 0.1864 - accuracy: 0.9191 - val_loss: 0.3416 - val_accuracy: 0.8713
Epoch 281/1000
470/470 [==============================] - 0s 87us/step - loss: 0.2024 - accuracy: 0.9255 - val_loss: 0.4126 - val_accuracy: 0.8416
Epoch 282/1000
470/470 [==============================] - 0s 85us/step - loss: 0.1934 - accuracy: 0.9106 - val_loss: 0.3495 - val_accuracy: 0.8812
Epoch 283/1000
470/470 [==============================] - 0s 83us/step - loss: 0.2092 - accuracy: 0.9170 - val_loss: 0.3737 - val_accuracy: 0.8564
Epoch 284/1000
470/470 [==============================] - 0s 82us/step - loss: 0.1889 - accuracy: 0.9255 - val_loss: 0

Epoch 334/1000
470/470 [==============================] - 0s 89us/step - loss: 0.1730 - accuracy: 0.9213 - val_loss: 0.3558 - val_accuracy: 0.8614
Epoch 335/1000
470/470 [==============================] - 0s 91us/step - loss: 0.1718 - accuracy: 0.9128 - val_loss: 0.3479 - val_accuracy: 0.8713
Epoch 336/1000
470/470 [==============================] - 0s 88us/step - loss: 0.1721 - accuracy: 0.9191 - val_loss: 0.3409 - val_accuracy: 0.8762
Epoch 337/1000
470/470 [==============================] - 0s 86us/step - loss: 0.1718 - accuracy: 0.9255 - val_loss: 0.4195 - val_accuracy: 0.8465
Epoch 338/1000
470/470 [==============================] - 0s 84us/step - loss: 0.1806 - accuracy: 0.9191 - val_loss: 0.3522 - val_accuracy: 0.8812
Epoch 339/1000
470/470 [==============================] - 0s 85us/step - loss: 0.1994 - accuracy: 0.9106 - val_loss: 0.3647 - val_accuracy: 0.8614
Epoch 340/1000
470/470 [==============================] - 0s 100us/step - loss: 0.1688 - accuracy: 0.9362 - val_loss: 

...

Epoch 669/1000
470/470 [==============================] - 0s 96us/step - loss: 0.1612 - accuracy: 0.9298 - val_loss: 0.4234 - val_accuracy: 0.8564
Epoch 670/1000
470/470 [==============================] - 0s 89us/step - loss: 0.1556 - accuracy: 0.9234 - val_loss: 0.3771 - val_accuracy: 0.8614
Epoch 671/1000
470/470 [==============================] - 0s 91us/step - loss: 0.1578 - accuracy: 0.9277 - val_loss: 0.4341 - val_accuracy: 0.8663
Epoch 672/1000
470/470 [==============================] - 0s 84us/step - loss: 0.1593 - accuracy: 0.9298 - val_loss: 0.3862 - val_accuracy: 0.8713
Epoch 673/1000
470/470 [==============================] - 0s 83us/step - loss: 0.1570 - accuracy: 0.9170 - val_loss: 0.3692 - val_accuracy: 0.8713
Epoch 674/1000
470/470 [==============================] - 0s 91us/step - loss: 0.1579 - accuracy: 0.9298 - val_loss: 0.4232 - val_accuracy: 0.8663
Epoch 675/1000
470/470 [==============================] - 0s 85us/step - loss: 0.1504 - accuracy: 0.9298 - val_loss: 0

Epoch 725/1000
470/470 [==============================] - 0s 95us/step - loss: 0.1744 - accuracy: 0.9191 - val_loss: 0.4035 - val_accuracy: 0.8663
Epoch 726/1000
470/470 [==============================] - 0s 100us/step - loss: 0.1483 - accuracy: 0.9298 - val_loss: 0.4056 - val_accuracy: 0.8713
Epoch 727/1000
470/470 [==============================] - 0s 111us/step - loss: 0.1489 - accuracy: 0.9298 - val_loss: 0.4005 - val_accuracy: 0.8663
Epoch 728/1000
470/470 [==============================] - 0s 98us/step - loss: 0.1491 - accuracy: 0.9298 - val_loss: 0.3967 - val_accuracy: 0.8713
Epoch 729/1000
470/470 [==============================] - 0s 94us/step - loss: 0.1544 - accuracy: 0.9298 - val_loss: 0.4410 - val_accuracy: 0.8564
Epoch 730/1000
470/470 [==============================] - 0s 94us/step - loss: 0.1699 - accuracy: 0.8957 - val_loss: 0.3826 - val_accuracy: 0.8762
Epoch 731/1000
470/470 [==============================] - 0s 87us/step - loss: 0.1702 - accuracy: 0.9362 - val_loss:

Epoch 781/1000
470/470 [==============================] - 0s 87us/step - loss: 0.1525 - accuracy: 0.9213 - val_loss: 0.4270 - val_accuracy: 0.8564
Epoch 782/1000
470/470 [==============================] - 0s 85us/step - loss: 0.1537 - accuracy: 0.9234 - val_loss: 0.4460 - val_accuracy: 0.8663
Epoch 783/1000
470/470 [==============================] - 0s 84us/step - loss: 0.1552 - accuracy: 0.9234 - val_loss: 0.3826 - val_accuracy: 0.8713
Epoch 784/1000
470/470 [==============================] - 0s 84us/step - loss: 0.1885 - accuracy: 0.9298 - val_loss: 0.4521 - val_accuracy: 0.8564
Epoch 785/1000
470/470 [==============================] - 0s 84us/step - loss: 0.1661 - accuracy: 0.9191 - val_loss: 0.3996 - val_accuracy: 0.8713
Epoch 786/1000
470/470 [==============================] - 0s 91us/step - loss: 0.1539 - accuracy: 0.9298 - val_loss: 0.3964 - val_accuracy: 0.8663
Epoch 787/1000
470/470 [==============================] - 0s 88us/step - loss: 0.1496 - accuracy: 0.9298 - val_loss: 0

Epoch 837/1000
470/470 [==============================] - 0s 102us/step - loss: 0.1545 - accuracy: 0.9213 - val_loss: 0.4041 - val_accuracy: 0.8713
Epoch 838/1000
470/470 [==============================] - 0s 81us/step - loss: 0.1545 - accuracy: 0.9298 - val_loss: 0.4442 - val_accuracy: 0.8614
Epoch 839/1000
470/470 [==============================] - 0s 86us/step - loss: 0.1543 - accuracy: 0.9298 - val_loss: 0.4127 - val_accuracy: 0.8663
Epoch 840/1000
470/470 [==============================] - 0s 90us/step - loss: 0.1475 - accuracy: 0.9213 - val_loss: 0.4241 - val_accuracy: 0.8564
Epoch 841/1000
470/470 [==============================] - 0s 87us/step - loss: 0.1493 - accuracy: 0.9213 - val_loss: 0.4274 - val_accuracy: 0.8663
Epoch 842/1000
470/470 [==============================] - 0s 82us/step - loss: 0.1503 - accuracy: 0.9298 - val_loss: 0.4162 - val_accuracy: 0.8663
Epoch 843/1000
470/470 [==============================] - 0s 88us/step - loss: 0.1530 - accuracy: 0.9298 - val_loss: 

Epoch 948/1000
470/470 [==============================] - 0s 86us/step - loss: 0.1514 - accuracy: 0.9298 - val_loss: 0.4347 - val_accuracy: 0.8663
Epoch 949/1000
470/470 [==============================] - 0s 91us/step - loss: 0.1510 - accuracy: 0.9234 - val_loss: 0.4419 - val_accuracy: 0.8564
Epoch 950/1000
470/470 [==============================] - 0s 94us/step - loss: 0.1508 - accuracy: 0.9213 - val_loss: 0.4357 - val_accuracy: 0.8663
Epoch 951/1000
470/470 [==============================] - 0s 87us/step - loss: 0.1476 - accuracy: 0.9213 - val_loss: 0.4460 - val_accuracy: 0.8663
Epoch 952/1000
470/470 [==============================] - 0s 94us/step - loss: 0.1476 - accuracy: 0.9298 - val_loss: 0.4371 - val_accuracy: 0.8663
Epoch 953/1000
470/470 [==============================] - 0s 89us/step - loss: 0.1513 - accuracy: 0.9213 - val_loss: 0.4439 - val_accuracy: 0.8564
Epoch 954/1000
470/470 [==============================] - 0s 94us/step - loss: 0.1480 - accuracy: 0.9340 - val_loss: 0

In [34]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

202/202 [==============================] - 0s 729us/step
over-sampling test accuracy: 87.62%


In [21]:
pred = model1_over.predict_classes(X_test_over)
pred

array([1, 0, 1, 1, 2, 1, 1, 1, 2, 0, 1, 0, 1, 0, 1, 2, 0, 0, 2, 2, 2, 0,
       0, 1, 1, 1, 1, 2, 2, 1, 2, 2, 0, 2, 1, 0, 0, 2, 1, 0, 2, 1, 2, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 2, 2, 0, 1, 0, 1,
       1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 1, 2, 2, 0, 2, 1, 2, 2, 1, 1, 0,
       2, 1, 2, 2, 1, 1, 2, 2, 1, 1, 0, 0, 1, 2, 0, 1, 2, 1, 2, 2, 1, 1,
       1, 1, 2, 0, 2, 0, 0, 0, 2, 0, 1, 0, 1, 2, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 2, 2, 1, 0, 2, 0, 0, 2, 2, 2, 2, 0, 2, 1, 2, 2, 2, 2, 1, 0, 2,
       1, 0, 2, 1, 0, 0, 2, 0, 2, 0, 2, 1, 2, 1, 2, 1, 0, 2, 1, 1, 2, 0,
       1, 2, 2, 1, 0, 2, 2, 2, 1, 1, 1, 2, 0, 0, 1, 1, 2, 0, 1, 0, 1, 1,
       1, 0, 1, 2])

In [22]:
dat['pred'] = pred
dat

,0,test,pred
0,NRS265,1,1
1,GA984,0,0
2,NRS119,0,1
3,NRS249,1,1
4,NRS255,2,2
...,...,...,...
197,NRS035,1,1
198,NRS387,1,1
199,NRS222,0,0
200,BCH-SA-11,1,1


In [23]:
proba1 = model1_over.predict_proba(X_test_over)
dat_proba1 = pd.DataFrame(proba1)

In [24]:
dat_proba1

,0,1,2
0,8.269686e-06,0.999988,3.901171e-06
1,7.964178e-01,0.203568,1.405292e-05
2,4.308351e-01,0.569159,5.776037e-06
3,4.446263e-03,0.995548,5.841435e-06
4,3.370932e-15,0.000043,9.999565e-01
...,...,...,...
197,4.782814e-07,0.999998,1.651596e-06
198,1.618924e-04,0.999815,2.323157e-05
199,7.844006e-01,0.215598,9.592115e-07
200,4.580007e-01,0.541993,6.226933e-06


In [25]:
dat_proba1.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba1.csv", index = False,
         header=None)

In [26]:
dat.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/1p003.csv", index = False,
         header=None)

In [40]:
hist1_over = model1_over.fit(X_train_over, y_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 470 samples, validate on 202 samples
Epoch 1/1000
470/470 [==============================] - 0s 329us/step - loss: 1.0320 - accuracy: 0.4936 - val_loss: 0.7991 - val_accuracy: 0.7079
Epoch 2/1000
470/470 [==============================] - 0s 90us/step - loss: 0.7637 - accuracy: 0.7277 - val_loss: 0.6552 - val_accuracy: 0.7822
Epoch 3/1000
470/470 [==============================] - 0s 87us/step - loss: 0.6579 - accuracy: 0.7043 - val_loss: 0.5800 - val_accuracy: 0.7525
Epoch 4/1000
470/470 [==============================] - 0s 94us/step - loss: 0.5812 - accuracy: 0.7638 - val_loss: 0.5205 - val_accuracy: 0.7970
Epoch 5/1000
470/470 [==============================] - 0s 86us/step - loss: 0.5275 - accuracy: 0.8021 - val_loss: 0.4837 - val_accuracy: 0.8069
Epoch 6/1000
470/470 [==============================] - 0s 95us/step - loss: 0.4715 - accuracy: 0.8106 - val_loss: 0.4633 - val_accuracy: 0.7772
Epoch 7/1000
470/470 [==============================] - 0s 94us/step - loss: 0.4454

Epoch 57/1000
470/470 [==============================] - 0s 81us/step - loss: 0.2396 - accuracy: 0.9000 - val_loss: 0.3507 - val_accuracy: 0.8515
Epoch 58/1000
470/470 [==============================] - 0s 83us/step - loss: 0.2394 - accuracy: 0.8851 - val_loss: 0.3304 - val_accuracy: 0.8713
Epoch 59/1000
470/470 [==============================] - 0s 91us/step - loss: 0.2357 - accuracy: 0.9149 - val_loss: 0.3320 - val_accuracy: 0.8515
Epoch 60/1000
470/470 [==============================] - 0s 102us/step - loss: 0.2303 - accuracy: 0.9021 - val_loss: 0.3419 - val_accuracy: 0.8614
Epoch 61/1000
470/470 [==============================] - 0s 172us/step - loss: 0.2373 - accuracy: 0.8979 - val_loss: 0.3312 - val_accuracy: 0.8564
Epoch 62/1000
470/470 [==============================] - 0s 125us/step - loss: 0.2260 - accuracy: 0.9191 - val_loss: 0.3405 - val_accuracy: 0.8614
Epoch 63/1000
470/470 [==============================] - 0s 116us/step - loss: 0.2246 - accuracy: 0.9149 - val_loss: 0.32

Epoch 113/1000
470/470 [==============================] - 0s 81us/step - loss: 0.1977 - accuracy: 0.9234 - val_loss: 0.3296 - val_accuracy: 0.8713
Epoch 114/1000
470/470 [==============================] - 0s 89us/step - loss: 0.1957 - accuracy: 0.9191 - val_loss: 0.3411 - val_accuracy: 0.8663
Epoch 115/1000
470/470 [==============================] - 0s 88us/step - loss: 0.2017 - accuracy: 0.9213 - val_loss: 0.3484 - val_accuracy: 0.8614
Epoch 116/1000
470/470 [==============================] - 0s 83us/step - loss: 0.1999 - accuracy: 0.9085 - val_loss: 0.3359 - val_accuracy: 0.8713
Epoch 117/1000
470/470 [==============================] - 0s 82us/step - loss: 0.2027 - accuracy: 0.9213 - val_loss: 0.3296 - val_accuracy: 0.8663
Epoch 118/1000
470/470 [==============================] - 0s 83us/step - loss: 0.1924 - accuracy: 0.9213 - val_loss: 0.3367 - val_accuracy: 0.8663
Epoch 119/1000
470/470 [==============================] - 0s 84us/step - loss: 0.1996 - accuracy: 0.9043 - val_loss: 0

470/470 [==============================] - 0s 101us/step - loss: 0.1723 - accuracy: 0.9255 - val_loss: 0.3550 - val_accuracy: 0.8663
Epoch 224/1000
470/470 [==============================] - 0s 94us/step - loss: 0.1714 - accuracy: 0.9149 - val_loss: 0.3615 - val_accuracy: 0.8663
Epoch 225/1000
470/470 [==============================] - 0s 100us/step - loss: 0.1749 - accuracy: 0.9255 - val_loss: 0.3738 - val_accuracy: 0.8663
Epoch 226/1000
470/470 [==============================] - 0s 101us/step - loss: 0.1707 - accuracy: 0.9213 - val_loss: 0.3771 - val_accuracy: 0.8663
Epoch 227/1000
470/470 [==============================] - 0s 126us/step - loss: 0.1795 - accuracy: 0.9213 - val_loss: 0.3403 - val_accuracy: 0.8762
Epoch 228/1000
470/470 [==============================] - 0s 111us/step - loss: 0.1833 - accuracy: 0.9255 - val_loss: 0.3806 - val_accuracy: 0.8564
Epoch 229/1000
470/470 [==============================] - 0s 106us/step - loss: 0.1745 - accuracy: 0.9234 - val_loss: 0.3649 - v

470/470 [==============================] - 0s 223us/step - loss: 0.1695 - accuracy: 0.9234 - val_loss: 0.4002 - val_accuracy: 0.8614
Epoch 334/1000
470/470 [==============================] - 0s 121us/step - loss: 0.1836 - accuracy: 0.9191 - val_loss: 0.4272 - val_accuracy: 0.8663
Epoch 335/1000
470/470 [==============================] - 0s 95us/step - loss: 0.1997 - accuracy: 0.9149 - val_loss: 0.3757 - val_accuracy: 0.8614
Epoch 336/1000
470/470 [==============================] - 0s 91us/step - loss: 0.1763 - accuracy: 0.9362 - val_loss: 0.4049 - val_accuracy: 0.8663
Epoch 337/1000
470/470 [==============================] - 0s 96us/step - loss: 0.1686 - accuracy: 0.9149 - val_loss: 0.4131 - val_accuracy: 0.8663
Epoch 338/1000
470/470 [==============================] - 0s 88us/step - loss: 0.1720 - accuracy: 0.9277 - val_loss: 0.3757 - val_accuracy: 0.8713
Epoch 339/1000
470/470 [==============================] - 0s 95us/step - loss: 0.1654 - accuracy: 0.9191 - val_loss: 0.4127 - val_a

...

470/470 [==============================] - 0s 94us/step - loss: 0.1485 - accuracy: 0.9298 - val_loss: 0.4827 - val_accuracy: 0.8663
Epoch 554/1000
470/470 [==============================] - 0s 97us/step - loss: 0.1563 - accuracy: 0.9298 - val_loss: 0.4705 - val_accuracy: 0.8663
Epoch 555/1000
470/470 [==============================] - 0s 98us/step - loss: 0.1511 - accuracy: 0.9170 - val_loss: 0.4710 - val_accuracy: 0.8713
Epoch 556/1000
470/470 [==============================] - 0s 92us/step - loss: 0.1525 - accuracy: 0.9298 - val_loss: 0.4534 - val_accuracy: 0.8713
Epoch 557/1000
470/470 [==============================] - 0s 92us/step - loss: 0.1624 - accuracy: 0.9213 - val_loss: 0.4739 - val_accuracy: 0.8713
Epoch 558/1000
470/470 [==============================] - 0s 106us/step - loss: 0.1518 - accuracy: 0.9298 - val_loss: 0.4821 - val_accuracy: 0.8663
Epoch 559/1000
470/470 [==============================] - 0s 106us/step - loss: 0.1518 - accuracy: 0.9298 - val_loss: 0.4479 - val_a

470/470 [==============================] - 0s 128us/step - loss: 0.1488 - accuracy: 0.9213 - val_loss: 0.5125 - val_accuracy: 0.8614
Epoch 664/1000
470/470 [==============================] - 0s 174us/step - loss: 0.1496 - accuracy: 0.9255 - val_loss: 0.5045 - val_accuracy: 0.8762
Epoch 665/1000
470/470 [==============================] - 0s 96us/step - loss: 0.1498 - accuracy: 0.9298 - val_loss: 0.5270 - val_accuracy: 0.8614
Epoch 666/1000
470/470 [==============================] - 0s 80us/step - loss: 0.1492 - accuracy: 0.9234 - val_loss: 0.5035 - val_accuracy: 0.8713
Epoch 667/1000
470/470 [==============================] - 0s 81us/step - loss: 0.1503 - accuracy: 0.9298 - val_loss: 0.4933 - val_accuracy: 0.8713
Epoch 668/1000
470/470 [==============================] - 0s 81us/step - loss: 0.1491 - accuracy: 0.9298 - val_loss: 0.4998 - val_accuracy: 0.8663
Epoch 669/1000
470/470 [==============================] - 0s 239us/step - loss: 0.1653 - accuracy: 0.9064 - val_loss: 0.5267 - val_

Epoch 719/1000
470/470 [==============================] - 0s 87us/step - loss: 0.1567 - accuracy: 0.9298 - val_loss: 0.5095 - val_accuracy: 0.8663
Epoch 720/1000
470/470 [==============================] - 0s 79us/step - loss: 0.1598 - accuracy: 0.9170 - val_loss: 0.5054 - val_accuracy: 0.8614
Epoch 721/1000
470/470 [==============================] - 0s 79us/step - loss: 0.1573 - accuracy: 0.9191 - val_loss: 0.5325 - val_accuracy: 0.8713
Epoch 722/1000
470/470 [==============================] - 0s 79us/step - loss: 0.1465 - accuracy: 0.9277 - val_loss: 0.5151 - val_accuracy: 0.8614
Epoch 723/1000
470/470 [==============================] - 0s 78us/step - loss: 0.1561 - accuracy: 0.9234 - val_loss: 0.5334 - val_accuracy: 0.8713
Epoch 724/1000
470/470 [==============================] - 0s 100us/step - loss: 0.1494 - accuracy: 0.9277 - val_loss: 0.5086 - val_accuracy: 0.8614
Epoch 725/1000
470/470 [==============================] - 0s 81us/step - loss: 0.1526 - accuracy: 0.9255 - val_loss: 

470/470 [==============================] - 0s 91us/step - loss: 0.1566 - accuracy: 0.9298 - val_loss: 0.5228 - val_accuracy: 0.8663
Epoch 830/1000
470/470 [==============================] - 0s 89us/step - loss: 0.1485 - accuracy: 0.9298 - val_loss: 0.5283 - val_accuracy: 0.8614
Epoch 831/1000
470/470 [==============================] - 0s 92us/step - loss: 0.1485 - accuracy: 0.9319 - val_loss: 0.5035 - val_accuracy: 0.8812
Epoch 832/1000
470/470 [==============================] - 0s 94us/step - loss: 0.1667 - accuracy: 0.9277 - val_loss: 0.5155 - val_accuracy: 0.8663
Epoch 833/1000
470/470 [==============================] - 0s 95us/step - loss: 0.1527 - accuracy: 0.9255 - val_loss: 0.5115 - val_accuracy: 0.8614
Epoch 834/1000
470/470 [==============================] - 0s 86us/step - loss: 0.1504 - accuracy: 0.9213 - val_loss: 0.5114 - val_accuracy: 0.8614
Epoch 835/1000
470/470 [==============================] - 0s 83us/step - loss: 0.1492 - accuracy: 0.9170 - val_loss: 0.5150 - val_acc

470/470 [==============================] - 0s 101us/step - loss: 0.1511 - accuracy: 0.9298 - val_loss: 0.5290 - val_accuracy: 0.8762
Epoch 940/1000
470/470 [==============================] - 0s 98us/step - loss: 0.1518 - accuracy: 0.9234 - val_loss: 0.5552 - val_accuracy: 0.8614
Epoch 941/1000
470/470 [==============================] - 0s 115us/step - loss: 0.1559 - accuracy: 0.9234 - val_loss: 0.5144 - val_accuracy: 0.8663
Epoch 942/1000
470/470 [==============================] - 0s 107us/step - loss: 0.1488 - accuracy: 0.9170 - val_loss: 0.5559 - val_accuracy: 0.8614
Epoch 943/1000
470/470 [==============================] - 0s 100us/step - loss: 0.1471 - accuracy: 0.9298 - val_loss: 0.5296 - val_accuracy: 0.8713
Epoch 944/1000
470/470 [==============================] - 0s 102us/step - loss: 0.1465 - accuracy: 0.9319 - val_loss: 0.5753 - val_accuracy: 0.8614
Epoch 945/1000
470/470 [==============================] - 0s 105us/step - loss: 0.1493 - accuracy: 0.9213 - val_loss: 0.5572 - v

In [41]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over.history['accuracy'])*100))

over-sampling train accuracy: 91.99%


In [71]:
df_proba = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [72]:
df_proba

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,CFBRSa26,0,0,0.758914,0.241086,4.638713e-07
1,p002ykpresabs_qual,NRS109,2,2,0.005361,0.016236,9.784034e-01
2,p002ykpresabs_qual,NRS112,0,0,0.726623,0.273376,1.520979e-06
3,p002ykpresabs_qual,NRS216,1,1,0.138322,0.861665,1.334123e-05
4,p002ykpresabs_qual,NRS021,0,0,0.882176,0.117824,1.414530e-10
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS148,2,2,0.000007,0.000099,9.998934e-01
4280,pyopresabsSTCC_qual,NRS255,2,2,0.000257,0.002048,9.976944e-01
4281,pyopresabsSTCC_qual,NRS205,2,2,0.000011,0.000045,9.999435e-01
4282,pyopresabsSTCC_qual,NRS255,2,2,0.000257,0.002048,9.976944e-01


In [73]:
y_prob = df_proba[df_proba['phage']=='p003pkpresabs_qual'].iloc[:,-3:]
y_prob = y_prob.to_numpy()
y_prob

array([[8.26968600e-06, 9.99987840e-01, 3.90117070e-06],
       [7.96417830e-01, 2.03568090e-01, 1.40529160e-05],
       [4.30835070e-01, 5.69159150e-01, 5.77603700e-06],
       [4.44626340e-03, 9.95547900e-01, 5.84143530e-06],
       [3.37093210e-15, 4.34801200e-05, 9.99956500e-01],
       [1.36448080e-07, 9.99999900e-01, 4.70130270e-12],
       [6.66015000e-05, 9.99645800e-01, 2.87512320e-04],
       [5.50760560e-05, 9.99940500e-01, 4.42293050e-06],
       [3.37093210e-15, 4.34801200e-05, 9.99956500e-01],
       [7.96417830e-01, 2.03568090e-01, 1.40529160e-05],
       [9.61630200e-03, 9.90382250e-01, 1.39389480e-06],
       [9.59891100e-01, 4.01088860e-02, 1.37021270e-08],
       [1.61892400e-04, 9.99814900e-01, 2.32315740e-05],
       [9.99807400e-01, 1.92572390e-04, 6.29983150e-11],
       [1.35447810e-05, 9.99986300e-01, 1.73022760e-07],
       [3.37093210e-15, 4.34801200e-05, 9.99956500e-01],
       [9.83839500e-01, 1.61601200e-02, 3.90576560e-07],
       [7.96417830e-01, 2.03568

In [74]:
## Retrieved from https://github.com/scikit-learn/scikit-learn/issues/3298
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer

def rocauc_ovo(truth, pred, average="macro", multi_class="ovo"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   
    
    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [75]:
ovo1 = rocauc_ovo(y_test_over, y_prob, average="macro", multi_class="ovo")
ovo1

0.9629998157361341

In [76]:
def rocauc_ovr(truth, pred, average="macro", multi_class="ovr"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   

    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [77]:
ovr1 = rocauc_ovr(y_test_over, y_prob, average="macro", multi_class="ovr")
ovr1

0.9629998157361341

In [78]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=234,
                                                    stratify=y_over)

In [79]:
dat2 = pd.DataFrame(X_test_over[:,0])
dat2['test'] = y_test_over

In [80]:
dat2

,0,test
0,CFBRSa05,0
1,NRS114,0
2,NRS168,1
3,NRS255,2
4,NRS209,2
...,...,...
197,NRS196,0
198,NRS255,2
199,NRS249,1
200,NRS209,2


In [81]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [29]:
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [32]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [33]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=64, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 470 samples, validate on 202 samples
Epoch 1/100
470/470 [==============================] - 0s 309us/step - loss: 1.0416 - accuracy: 0.4723 - val_loss: 0.7918 - val_accuracy: 0.5891
Epoch 2/100
470/470 [==============================] - 0s 135us/step - loss: 0.7368 - accuracy: 0.6957 - val_loss: 0.6944 - val_accuracy: 0.7772
Epoch 3/100
470/470 [==============================] - 0s 144us/step - loss: 0.6484 - accuracy: 0.7702 - val_loss: 0.6404 - val_accuracy: 0.7822
Epoch 4/100
470/470 [==============================] - 0s 239us/step - loss: 0.5929 - accuracy: 0.7766 - val_loss: 0.6098 - val_accuracy: 0.7327
Epoch 5/100
470/470 [==============================] - 0s 207us/step - loss: 0.5510 - accuracy: 0.7809 - val_loss: 0.5715 - val_accuracy: 0.7624
Epoch 6/100
470/470 [==============================] - 0s 224us/step - loss: 0.5229 - accuracy: 0.7745 - val_loss: 0.5497 - val_accuracy: 0.7921
Epoch 7/100
470/470 [==============================] - 0s 161us/step - loss: 0.5053 

Epoch 57/100
470/470 [==============================] - 0s 138us/step - loss: 0.2526 - accuracy: 0.9021 - val_loss: 0.3761 - val_accuracy: 0.8515
Epoch 58/100
470/470 [==============================] - 0s 88us/step - loss: 0.2546 - accuracy: 0.9021 - val_loss: 0.3779 - val_accuracy: 0.8515
Epoch 59/100
470/470 [==============================] - 0s 98us/step - loss: 0.2536 - accuracy: 0.8936 - val_loss: 0.3815 - val_accuracy: 0.8515
Epoch 60/100
470/470 [==============================] - 0s 104us/step - loss: 0.2444 - accuracy: 0.9064 - val_loss: 0.3647 - val_accuracy: 0.8515
Epoch 61/100
470/470 [==============================] - 0s 87us/step - loss: 0.2479 - accuracy: 0.8915 - val_loss: 0.3676 - val_accuracy: 0.8515
Epoch 62/100
470/470 [==============================] - 0s 113us/step - loss: 0.2429 - accuracy: 0.9043 - val_loss: 0.3698 - val_accuracy: 0.8515
Epoch 63/100
470/470 [==============================] - 0s 88us/step - loss: 0.2533 - accuracy: 0.9021 - val_loss: 0.4325 - val

In [65]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

202/202 [==============================] - 0s 104us/step
over-sampling test accuracy: 87.13%


In [34]:
pred2 = model1_over2.predict_classes(X_test_over)
pred2

array([0, 1, 1, 2, 2, 2, 2, 1, 2, 2, 0, 1, 0, 0, 2, 1, 0, 2, 1, 2, 2, 2,
       0, 2, 0, 0, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 2, 1, 1, 0,
       2, 0, 0, 2, 2, 2, 1, 0, 0, 2, 2, 1, 1, 2, 1, 2, 0, 0, 1, 1, 1, 1,
       1, 2, 1, 2, 0, 1, 2, 1, 0, 0, 1, 1, 2, 1, 1, 2, 1, 1, 1, 0, 1, 1,
       1, 2, 0, 1, 2, 1, 1, 1, 0, 2, 1, 1, 1, 1, 0, 2, 2, 1, 0, 1, 1, 0,
       2, 0, 1, 1, 1, 2, 0, 2, 1, 0, 0, 2, 1, 1, 1, 1, 1, 0, 0, 2, 1, 1,
       1, 0, 1, 1, 1, 1, 2, 2, 1, 2, 0, 2, 2, 0, 1, 2, 1, 1, 0, 2, 2, 2,
       0, 2, 1, 1, 0, 2, 2, 2, 0, 1, 0, 2, 1, 2, 0, 0, 2, 0, 2, 2, 2, 1,
       1, 1, 2, 1, 2, 1, 1, 0, 1, 2, 1, 2, 1, 1, 0, 0, 1, 1, 2, 2, 1, 0,
       2, 1, 2, 0])

In [35]:
dat2['pred'] = pred2
dat2

,0,test,pred
0,CFBRSa05,0,0
1,NRS114,0,1
2,NRS168,1,1
3,NRS255,2,2
4,NRS209,2,2
...,...,...,...
197,NRS196,0,0
198,NRS255,2,2
199,NRS249,1,1
200,NRS209,2,2


In [36]:
proba2 = model1_over2.predict_proba(X_test_over)
dat_proba2 = pd.DataFrame(proba2)

In [37]:
dat_proba2

,0,1,2
0,0.748078,0.251744,1.774990e-04
1,0.400403,0.599597,1.638532e-09
2,0.080697,0.918530,7.735260e-04
3,0.000411,0.002266,9.973239e-01
4,0.000051,0.000014,9.999348e-01
...,...,...,...
197,0.842184,0.157630,1.863075e-04
198,0.000411,0.002266,9.973239e-01
199,0.035768,0.964232,2.466800e-09
200,0.000051,0.000014,9.999348e-01


In [38]:
dat_proba2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba2.csv", index = False,
         header=None)

In [39]:
dat2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/2p003.csv", index = False,
         header=None)

In [69]:
hist1_over2 = model1_over2.fit(X_train_over, y_train_over,
          batch_size=64, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 470 samples, validate on 202 samples
Epoch 1/100
470/470 [==============================] - 0s 146us/step - loss: 0.1951 - accuracy: 0.9106 - val_loss: 0.3313 - val_accuracy: 0.8812
Epoch 2/100
470/470 [==============================] - 0s 101us/step - loss: 0.2150 - accuracy: 0.9191 - val_loss: 0.4003 - val_accuracy: 0.8465
Epoch 3/100
470/470 [==============================] - 0s 93us/step - loss: 0.2037 - accuracy: 0.9170 - val_loss: 0.3918 - val_accuracy: 0.8614
Epoch 4/100
470/470 [==============================] - 0s 86us/step - loss: 0.1894 - accuracy: 0.9149 - val_loss: 0.3288 - val_accuracy: 0.8812
Epoch 5/100
470/470 [==============================] - 0s 93us/step - loss: 0.1911 - accuracy: 0.9128 - val_loss: 0.3997 - val_accuracy: 0.8614
Epoch 6/100
470/470 [==============================] - 0s 95us/step - loss: 0.1901 - accuracy: 0.9234 - val_loss: 0.3690 - val_accuracy: 0.8713
Epoch 7/100
470/470 [==============================] - 0s 137us/step - loss: 0.1819 - ac

470/470 [==============================] - 0s 91us/step - loss: 0.1740 - accuracy: 0.9213 - val_loss: 0.3526 - val_accuracy: 0.8564
Epoch 58/100
470/470 [==============================] - 0s 91us/step - loss: 0.1756 - accuracy: 0.9191 - val_loss: 0.3619 - val_accuracy: 0.8663
Epoch 59/100
470/470 [==============================] - 0s 101us/step - loss: 0.1818 - accuracy: 0.9234 - val_loss: 0.4214 - val_accuracy: 0.8465
Epoch 60/100
470/470 [==============================] - 0s 100us/step - loss: 0.1967 - accuracy: 0.9149 - val_loss: 0.3299 - val_accuracy: 0.8812
Epoch 61/100
470/470 [==============================] - 0s 87us/step - loss: 0.2105 - accuracy: 0.9234 - val_loss: 0.4678 - val_accuracy: 0.8366
Epoch 62/100
470/470 [==============================] - 0s 107us/step - loss: 0.1944 - accuracy: 0.9191 - val_loss: 0.3541 - val_accuracy: 0.8713
Epoch 63/100
470/470 [==============================] - 0s 97us/step - loss: 0.1773 - accuracy: 0.9298 - val_loss: 0.3501 - val_accuracy: 0.

In [70]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over2.history['accuracy'])*100))

over-sampling train accuracy: 91.84%


In [82]:
df_proba2 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [83]:
df_proba2

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS148,2,2,0.000056,1.748042e-03,9.981960e-01
1,p002ykpresabs_qual,BCH-SA-03,1,0,0.712007,2.879924e-01,9.646217e-07
2,p002ykpresabs_qual,NRS218,1,1,0.006222,9.937732e-01,4.482882e-06
3,p002ykpresabs_qual,NRS036,0,0,0.882617,1.173831e-01,2.310933e-10
4,p002ykpresabs_qual,NRS386,1,0,0.571179,4.288184e-01,2.444667e-06
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS112,1,1,0.001860,9.979747e-01,1.653396e-04
4280,pyopresabsSTCC_qual,SR1065,0,0,0.982940,1.705227e-02,7.349168e-06
4281,pyopresabsSTCC_qual,NRS203,0,0,0.997093,1.962516e-03,9.441347e-04
4282,pyopresabsSTCC_qual,CFBREBSa129,0,0,1.000000,3.031141e-13,3.208205e-09


In [84]:
y_prob2 = df_proba2[df_proba2['phage']=='p003pkpresabs_qual'].iloc[:,-3:]
y_prob2 = y_prob2.to_numpy()
y_prob2

array([[7.4807847e-01, 2.5174406e-01, 1.7749900e-04],
       [4.0040302e-01, 5.9959704e-01, 1.6385315e-09],
       [8.0696730e-02, 9.1852975e-01, 7.7352600e-04],
       [4.1052580e-04, 2.2656214e-03, 9.9732390e-01],
       [5.1128540e-05, 1.4095375e-05, 9.9993480e-01],
       [4.1052580e-04, 2.2656214e-03, 9.9732390e-01],
       [5.1128540e-05, 1.4095375e-05, 9.9993480e-01],
       [1.5061130e-03, 9.9825317e-01, 2.4073607e-04],
       [5.1128540e-05, 1.4095375e-05, 9.9993480e-01],
       [6.9435990e-04, 5.1369607e-02, 9.4793600e-01],
       [7.4807847e-01, 2.5174406e-01, 1.7749900e-04],
       [2.4884485e-03, 9.9725795e-01, 2.5355126e-04],
       [9.8760134e-01, 1.2388925e-02, 9.7060240e-06],
       [9.7712880e-01, 2.2741630e-02, 1.2955115e-04],
       [4.1052580e-04, 2.2656214e-03, 9.9732390e-01],
       [8.0696730e-02, 9.1852975e-01, 7.7352600e-04],
       [7.4807847e-01, 2.5174406e-01, 1.7749900e-04],
       [4.1052580e-04, 2.2656214e-03, 9.9732390e-01],
       [1.5467422e-02, 9.845

In [85]:
ovo2 = rocauc_ovo(y_test_over, y_prob2, average="macro", multi_class="ovo")
ovo2

0.9511297000834608

In [86]:
ovr2 = rocauc_ovr(y_test_over, y_prob2, average="macro", multi_class="ovr")
ovr2

0.9511297000834608

In [111]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=345,
                                                    stratify=y_over)

In [112]:
dat3 = pd.DataFrame(X_test_over[:,0])
dat3['test'] = y_test_over

In [113]:
dat3

,0,test
0,NRS148,2
1,NRS209,2
2,NRS187,1
3,CFBREBSa116,0
4,NRS187,1
...,...,...
197,GA12,0
198,NRS209,2
199,NRS265,1
200,NRS253,1


In [114]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [44]:
model1_over3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [45]:
model1_over3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [46]:
model1_over3.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 470 samples, validate on 202 samples
Epoch 1/100
470/470 [==============================] - 0s 339us/step - loss: 0.9308 - accuracy: 0.5638 - val_loss: 0.7459 - val_accuracy: 0.7228
Epoch 2/100
470/470 [==============================] - 0s 119us/step - loss: 0.6359 - accuracy: 0.7702 - val_loss: 0.6167 - val_accuracy: 0.7079
Epoch 3/100
470/470 [==============================] - 0s 135us/step - loss: 0.5036 - accuracy: 0.8043 - val_loss: 0.4985 - val_accuracy: 0.7673
Epoch 4/100
470/470 [==============================] - 0s 196us/step - loss: 0.4389 - accuracy: 0.7915 - val_loss: 0.4690 - val_accuracy: 0.7624
Epoch 5/100
470/470 [==============================] - 0s 218us/step - loss: 0.4322 - accuracy: 0.8128 - val_loss: 0.4557 - val_accuracy: 0.7970
Epoch 6/100
470/470 [==============================] - 0s 211us/step - loss: 0.3943 - accuracy: 0.8277 - val_loss: 0.4305 - val_accuracy: 0.7921
Epoch 7/100
470/470 [==============================] - 0s 177us/step - loss: 0.3734 

In [79]:
acc_test_over3 = model1_over3.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over3*100))

202/202 [==============================] - 0s 95us/step
over-sampling test accuracy: 89.11%


In [47]:
pred3 = model1_over3.predict_classes(X_test_over)
pred3

array([2, 2, 1, 0, 1, 2, 2, 0, 2, 1, 1, 0, 2, 0, 0, 0, 0, 1, 2, 1, 0, 0,
       0, 0, 1, 0, 1, 2, 0, 2, 2, 0, 1, 0, 2, 2, 0, 1, 0, 0, 1, 0, 1, 2,
       0, 1, 1, 0, 1, 0, 0, 2, 2, 1, 0, 1, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2,
       2, 2, 1, 1, 0, 0, 2, 0, 0, 1, 0, 1, 2, 1, 1, 0, 0, 2, 1, 1, 1, 1,
       2, 2, 2, 0, 0, 1, 0, 1, 0, 2, 0, 1, 2, 2, 0, 2, 1, 2, 1, 1, 0, 0,
       0, 2, 1, 0, 0, 2, 1, 1, 0, 0, 0, 1, 0, 1, 1, 2, 1, 2, 0, 2, 0, 1,
       1, 1, 2, 2, 0, 0, 0, 2, 2, 2, 0, 2, 2, 2, 0, 0, 1, 0, 2, 1, 2, 0,
       0, 2, 1, 1, 1, 2, 1, 0, 0, 2, 2, 0, 0, 2, 2, 0, 2, 0, 0, 0, 2, 1,
       1, 2, 0, 1, 0, 1, 1, 2, 1, 2, 0, 2, 2, 2, 0, 0, 0, 2, 0, 1, 1, 0,
       2, 1, 1, 0])

In [48]:
dat3['pred'] = pred3
dat3

,0,test,pred
0,NRS148,2,2
1,NRS209,2,2
2,NRS187,1,1
3,CFBREBSa116,0,0
4,NRS187,1,1
...,...,...,...
197,GA12,0,0
198,NRS209,2,2
199,NRS265,1,1
200,NRS253,1,1


In [49]:
proba3 = model1_over3.predict_proba(X_test_over)
dat_proba3 = pd.DataFrame(proba3)

In [50]:
dat_proba3

,0,1,2
0,0.000191,0.011061,9.887473e-01
1,0.000054,0.000002,9.999443e-01
2,0.036562,0.963436,1.711485e-06
3,0.813265,0.186730,5.459639e-06
4,0.036562,0.963436,1.711485e-06
...,...,...,...
197,0.851234,0.148766,2.317629e-09
198,0.000054,0.000002,9.999443e-01
199,0.016951,0.982086,9.635856e-04
200,0.011996,0.987964,4.010012e-05


In [51]:
dat_proba3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba3.csv", index = False,
         header=None)

In [52]:
dat3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/3p003.csv", index = False,
         header=None)

In [83]:
hist1_over3 = model1_over3.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 470 samples, validate on 202 samples
Epoch 1/100
470/470 [==============================] - 0s 203us/step - loss: 0.1745 - accuracy: 0.9255 - val_loss: 0.3526 - val_accuracy: 0.8267
Epoch 2/100
470/470 [==============================] - 0s 178us/step - loss: 0.1947 - accuracy: 0.9191 - val_loss: 0.2971 - val_accuracy: 0.8861
Epoch 3/100
470/470 [==============================] - 0s 154us/step - loss: 0.1788 - accuracy: 0.9191 - val_loss: 0.2897 - val_accuracy: 0.8861
Epoch 4/100
470/470 [==============================] - 0s 155us/step - loss: 0.1786 - accuracy: 0.9234 - val_loss: 0.3008 - val_accuracy: 0.8515
Epoch 5/100
470/470 [==============================] - 0s 173us/step - loss: 0.1768 - accuracy: 0.9234 - val_loss: 0.3104 - val_accuracy: 0.8515
Epoch 6/100
470/470 [==============================] - 0s 127us/step - loss: 0.1834 - accuracy: 0.9255 - val_loss: 0.2843 - val_accuracy: 0.8911
Epoch 7/100
470/470 [==============================] - 0s 107us/step - loss: 0.1731 

Epoch 57/100
470/470 [==============================] - 0s 229us/step - loss: 0.1640 - accuracy: 0.9234 - val_loss: 0.2849 - val_accuracy: 0.8861
Epoch 58/100
470/470 [==============================] - 0s 376us/step - loss: 0.1777 - accuracy: 0.9191 - val_loss: 0.3225 - val_accuracy: 0.8465
Epoch 59/100
470/470 [==============================] - 0s 194us/step - loss: 0.1831 - accuracy: 0.9255 - val_loss: 0.3413 - val_accuracy: 0.8416
Epoch 60/100
470/470 [==============================] - 0s 113us/step - loss: 0.1831 - accuracy: 0.9149 - val_loss: 0.2707 - val_accuracy: 0.8861
Epoch 61/100
470/470 [==============================] - 0s 121us/step - loss: 0.1601 - accuracy: 0.9319 - val_loss: 0.2847 - val_accuracy: 0.8911
Epoch 62/100
470/470 [==============================] - 0s 109us/step - loss: 0.1737 - accuracy: 0.9255 - val_loss: 0.2694 - val_accuracy: 0.8960
Epoch 63/100
470/470 [==============================] - 0s 119us/step - loss: 0.1638 - accuracy: 0.9298 - val_loss: 0.3483 -

In [84]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over3.history['accuracy'])*100))

over-sampling train accuracy: 92.33%


In [115]:
df_proba3 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [116]:
df_proba3

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
1,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
2,p002ykpresabs_qual,NRS222,0,0,0.851725,0.148269,5.980786e-06
3,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
4,p002ykpresabs_qual,GA50245,0,0,0.812055,0.187945,1.161034e-07
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS255,2,2,0.000633,0.000928,9.984396e-01
4280,pyopresabsSTCC_qual,NRS255,2,2,0.000633,0.000928,9.984396e-01
4281,pyopresabsSTCC_qual,NRS266,1,1,0.025932,0.974061,7.323514e-06
4282,pyopresabsSTCC_qual,NRS001,1,1,0.000597,0.999403,3.675362e-10


In [117]:
y_prob3 = df_proba3[df_proba3['phage']=='p003pkpresabs_qual'].iloc[:,-3:]
y_prob3 = y_prob3.to_numpy()
y_prob3

array([[1.91267360e-04, 1.10614950e-02, 9.88747300e-01],
       [5.36823940e-05, 2.00586990e-06, 9.99944300e-01],
       [3.65622500e-02, 9.63436070e-01, 1.71148490e-06],
       [8.13264850e-01, 1.86729730e-01, 5.45963940e-06],
       [3.65622500e-02, 9.63436070e-01, 1.71148490e-06],
       [5.36823940e-05, 2.00586990e-06, 9.99944300e-01],
       [5.78180620e-05, 8.91463060e-05, 9.99853000e-01],
       [8.13264850e-01, 1.86729730e-01, 5.45963940e-06],
       [1.91267360e-04, 1.10614950e-02, 9.88747300e-01],
       [1.69506900e-02, 9.82085700e-01, 9.63585570e-04],
       [3.03381700e-02, 9.69647500e-01, 1.42874940e-05],
       [8.13264850e-01, 1.86729730e-01, 5.45963940e-06],
       [5.36823940e-05, 2.00586990e-06, 9.99944300e-01],
       [8.20870100e-01, 1.79116340e-01, 1.35026140e-05],
       [8.51234000e-01, 1.48765950e-01, 2.31762900e-09],
       [8.13264850e-01, 1.86729730e-01, 5.45963940e-06],
       [8.13264850e-01, 1.86729730e-01, 5.45963940e-06],
       [2.23315230e-02, 9.76742

In [118]:
ovo3 = rocauc_ovo(y_test_over, y_prob3, average="macro", multi_class="ovo")
ovo3

0.9545040321269469

In [119]:
ovr3 = rocauc_ovr(y_test_over, y_prob3, average="macro", multi_class="ovr")
ovr3

0.9545040321269469

In [98]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=456,
                                                    stratify=y_over)

In [99]:
dat4 = pd.DataFrame(X_test_over[:,0])
dat4['test'] = y_test_over

In [100]:
dat4

,0,test
0,CFBRSa04,0
1,NRS021,0
2,NRS073,0
3,NRS049,0
4,CA541,1
...,...,...
197,NRS387,1
198,SR1746,0
199,NRS148,2
200,NRS255,2


In [101]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [57]:
model1_over4 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [58]:
model1_over4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [59]:
model1_over4.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 470 samples, validate on 202 samples
Epoch 1/100
470/470 [==============================] - 0s 436us/step - loss: 0.8900 - accuracy: 0.5404 - val_loss: 0.6026 - val_accuracy: 0.7574
Epoch 2/100
470/470 [==============================] - 0s 155us/step - loss: 0.6172 - accuracy: 0.7255 - val_loss: 0.4638 - val_accuracy: 0.7970
Epoch 3/100
470/470 [==============================] - 0s 174us/step - loss: 0.4833 - accuracy: 0.7809 - val_loss: 0.4224 - val_accuracy: 0.8168
Epoch 4/100
470/470 [==============================] - 0s 143us/step - loss: 0.4603 - accuracy: 0.7787 - val_loss: 0.4005 - val_accuracy: 0.8564
Epoch 5/100
470/470 [==============================] - 0s 149us/step - loss: 0.4314 - accuracy: 0.8085 - val_loss: 0.3832 - val_accuracy: 0.8317
Epoch 6/100
470/470 [==============================] - 0s 154us/step - loss: 0.4080 - accuracy: 0.8234 - val_loss: 0.3685 - val_accuracy: 0.8317
Epoch 7/100
470/470 [==============================] - 0s 181us/step - loss: 0.3881 

Epoch 57/100
470/470 [==============================] - 0s 335us/step - loss: 0.2796 - accuracy: 0.8957 - val_loss: 0.2441 - val_accuracy: 0.8911
Epoch 58/100
470/470 [==============================] - 0s 283us/step - loss: 0.2373 - accuracy: 0.9149 - val_loss: 0.2267 - val_accuracy: 0.9158
Epoch 59/100
470/470 [==============================] - 0s 230us/step - loss: 0.2477 - accuracy: 0.8915 - val_loss: 0.2456 - val_accuracy: 0.8911
Epoch 60/100
470/470 [==============================] - 0s 238us/step - loss: 0.2300 - accuracy: 0.9064 - val_loss: 0.2269 - val_accuracy: 0.9158
Epoch 61/100
470/470 [==============================] - 0s 191us/step - loss: 0.2228 - accuracy: 0.9213 - val_loss: 0.2491 - val_accuracy: 0.8911
Epoch 62/100
470/470 [==============================] - 0s 169us/step - loss: 0.2226 - accuracy: 0.9170 - val_loss: 0.2377 - val_accuracy: 0.9059
Epoch 63/100
470/470 [==============================] - 0s 185us/step - loss: 0.2148 - accuracy: 0.9149 - val_loss: 0.2273 -

In [92]:
acc_test_over4 = model1_over4.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over4*100))

202/202 [==============================] - 0s 101us/step
over-sampling test accuracy: 89.60%


In [60]:
pred4 = model1_over4.predict_classes(X_test_over)
pred4

array([0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 2, 2, 2, 2, 1, 1, 2, 1, 2, 1,
       0, 1, 1, 0, 0, 2, 0, 1, 0, 0, 2, 0, 2, 2, 1, 1, 1, 1, 1, 2, 0, 0,
       1, 1, 2, 1, 0, 0, 1, 2, 0, 1, 2, 1, 1, 1, 2, 1, 0, 2, 2, 2, 0, 0,
       2, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 2, 1, 1, 0, 1, 1, 1, 2, 2, 0, 2,
       2, 1, 2, 0, 0, 2, 1, 1, 2, 2, 2, 1, 1, 2, 0, 2, 1, 1, 1, 2, 0, 1,
       0, 1, 2, 1, 1, 2, 0, 0, 2, 0, 0, 2, 1, 2, 1, 2, 0, 1, 0, 1, 1, 1,
       0, 1, 1, 0, 0, 1, 2, 0, 0, 1, 1, 0, 2, 1, 0, 0, 1, 1, 2, 2, 2, 2,
       1, 2, 1, 2, 0, 2, 1, 0, 2, 0, 1, 0, 1, 0, 2, 0, 0, 1, 2, 0, 2, 1,
       2, 2, 1, 2, 1, 2, 2, 0, 0, 2, 2, 0, 0, 2, 1, 1, 0, 2, 2, 2, 0, 1,
       0, 2, 2, 1])

In [61]:
dat4['pred'] = pred4
dat4

,0,test,pred
0,CFBRSa04,0,0
1,NRS021,0,1
2,NRS073,0,1
3,NRS049,0,0
4,CA541,1,0
...,...,...,...
197,NRS387,1,1
198,SR1746,0,0
199,NRS148,2,2
200,NRS255,2,2


In [62]:
proba4 = model1_over4.predict_proba(X_test_over)
dat_proba4 = pd.DataFrame(proba4)

In [63]:
dat_proba4

,0,1,2
0,0.726998,0.272999,2.558600e-06
1,0.278299,0.721701,2.888859e-08
2,0.278299,0.721701,2.888859e-08
3,0.726998,0.272999,2.558600e-06
4,0.726998,0.272999,2.558600e-06
...,...,...,...
197,0.044933,0.955058,8.977762e-06
198,0.812145,0.187846,9.036949e-06
199,0.000017,0.012262,9.877204e-01
200,0.000009,0.000246,9.997457e-01


In [64]:
dat_proba4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba4.csv", index = False,
         header=None)

In [65]:
dat4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/4p003.csv", index = False,
         header=None)

In [97]:
hist1_over4 = model1_over4.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 470 samples, validate on 202 samples
Epoch 1/100
470/470 [==============================] - 0s 280us/step - loss: 0.2112 - accuracy: 0.9085 - val_loss: 0.2078 - val_accuracy: 0.9158
Epoch 2/100
470/470 [==============================] - 0s 178us/step - loss: 0.1993 - accuracy: 0.9191 - val_loss: 0.2092 - val_accuracy: 0.9158
Epoch 3/100
470/470 [==============================] - 0s 130us/step - loss: 0.1940 - accuracy: 0.9170 - val_loss: 0.2183 - val_accuracy: 0.9158
Epoch 4/100
470/470 [==============================] - 0s 144us/step - loss: 0.2228 - accuracy: 0.9106 - val_loss: 0.2418 - val_accuracy: 0.9158
Epoch 5/100
470/470 [==============================] - 0s 143us/step - loss: 0.2234 - accuracy: 0.9128 - val_loss: 0.2197 - val_accuracy: 0.9158
Epoch 6/100
470/470 [==============================] - 0s 130us/step - loss: 0.1977 - accuracy: 0.9106 - val_loss: 0.2178 - val_accuracy: 0.9158
Epoch 7/100
470/470 [==============================] - 0s 139us/step - loss: 0.2279 

Epoch 57/100
470/470 [==============================] - 0s 110us/step - loss: 0.1968 - accuracy: 0.9085 - val_loss: 0.2123 - val_accuracy: 0.9158
Epoch 58/100
470/470 [==============================] - 0s 115us/step - loss: 0.1908 - accuracy: 0.9064 - val_loss: 0.2055 - val_accuracy: 0.9109
Epoch 59/100
470/470 [==============================] - 0s 108us/step - loss: 0.1859 - accuracy: 0.9106 - val_loss: 0.2070 - val_accuracy: 0.9109
Epoch 60/100
470/470 [==============================] - 0s 105us/step - loss: 0.1832 - accuracy: 0.9170 - val_loss: 0.2131 - val_accuracy: 0.9059
Epoch 61/100
470/470 [==============================] - 0s 112us/step - loss: 0.1852 - accuracy: 0.9106 - val_loss: 0.2087 - val_accuracy: 0.9059
Epoch 62/100
470/470 [==============================] - 0s 158us/step - loss: 0.2021 - accuracy: 0.9149 - val_loss: 0.2228 - val_accuracy: 0.9010
Epoch 63/100
470/470 [==============================] - 0s 142us/step - loss: 0.1996 - accuracy: 0.9106 - val_loss: 0.2486 -

In [98]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over4.history['accuracy'])*100))

over-sampling train accuracy: 91.18%


In [102]:
df_proba4 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [103]:
df_proba4

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS110,1,1,0.000003,0.999997,5.870196e-13
1,p002ykpresabs_qual,NRS216,1,1,0.039254,0.960745,9.078969e-07
2,p002ykpresabs_qual,NRS386,1,1,0.326752,0.673248,1.061032e-07
3,p002ykpresabs_qual,CFBRSa25,0,0,0.611084,0.388916,7.664974e-07
4,p002ykpresabs_qual,BCH-SA-03,1,0,0.611084,0.388916,7.664974e-07
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS236,1,1,0.000052,0.999768,1.803156e-04
4280,pyopresabsSTCC_qual,NRS029,0,1,0.322350,0.677496,1.533154e-04
4281,pyopresabsSTCC_qual,NRS148,2,2,0.000006,0.000026,9.999682e-01
4282,pyopresabsSTCC_qual,CFBRSa28,0,0,0.999288,0.000176,5.361527e-04


In [104]:
y_prob4 = df_proba4[df_proba4['phage']=='p003pkpresabs_qual'].iloc[:,-3:]
y_prob4 = y_prob4.to_numpy()
y_prob4

array([[7.26997900e-01, 2.72999470e-01, 2.55860030e-06],
       [2.78299300e-01, 7.21700700e-01, 2.88885950e-08],
       [2.78299300e-01, 7.21700700e-01, 2.88885950e-08],
       [7.26997900e-01, 2.72999470e-01, 2.55860030e-06],
       [7.26997900e-01, 2.72999470e-01, 2.55860030e-06],
       [2.78299300e-01, 7.21700700e-01, 2.88885950e-08],
       [2.45650400e-03, 9.97321800e-01, 2.21743730e-04],
       [3.31315600e-02, 9.66840100e-01, 2.82984580e-05],
       [7.26997900e-01, 2.72999470e-01, 2.55860030e-06],
       [7.26997900e-01, 2.72999470e-01, 2.55860030e-06],
       [5.79772060e-01, 4.20222500e-01, 5.50420240e-06],
       [5.74421500e-01, 4.25568820e-01, 9.75001100e-06],
       [8.81504800e-06, 2.45550360e-04, 9.99745670e-01],
       [5.68631540e-08, 8.62353300e-06, 9.99991300e-01],
       [8.81504800e-06, 2.45550360e-04, 9.99745670e-01],
       [5.68631540e-08, 8.62353300e-06, 9.99991300e-01],
       [8.24993100e-02, 9.17475940e-01, 2.48390900e-05],
       [4.09551380e-01, 5.90448

In [105]:
ovo4 = rocauc_ovo(y_test_over, y_prob4, average="macro", multi_class="ovo")
ovo4

0.9746283560411451

In [106]:
ovr4 = rocauc_ovr(y_test_over, y_prob4, average="macro", multi_class="ovr")
ovr4

0.9746283560411451

In [120]:
ovos = [ovo1, ovo2, ovo3, ovo4]
np.mean(ovos)

0.9608154759969217

In [121]:
np.std(ovos)

0.009072149331636208

In [122]:
ovrs = [ovr1, ovr2, ovr3, ovr4]
np.mean(ovrs)

0.9608154759969217

In [123]:
np.std(ovrs)

0.009072149331636208

In [99]:
accs = [acc_test_over, acc_test_over2, acc_test_over3, acc_test_over4]

In [100]:
mean = np.mean(accs)
print('over-sampling test accuracy mean: %.2f%%' % (mean*100))

over-sampling test accuracy mean: 88.37%


In [101]:
std = np.std(accs)
print('over-sampling test accuracy standard deviation:', std)

over-sampling test accuracy standard deviation: 0.010205707566638102


In [102]:
accs_train = [np.mean(hist1_over.history['accuracy']), np.mean(hist1_over2.history['accuracy']), np.mean(hist1_over3.history['accuracy']),
             np.mean(hist1_over4.history['accuracy'])]

In [103]:
mean_train = np.mean(accs_train)
print('over-sampling train accuracy mean: %.2f%%' % (mean_train*100))

over-sampling train accuracy mean: 91.83%


In [104]:
std_train = np.std(accs_train)
print('over-sampling train accuracy standard deviation:', std_train)

over-sampling train accuracy standard deviation: 0.004177694
